Step 1: extract files from folder into one large folder. When I downloaded the data from instagram each conversation was stored in a different folder. But we need all our conversations in one folder. So we will extract our .json files(the conversations) from their individual folders and put it in one.

In [ ]:
#importing libraries
import os
import shutil
import csv
import json
import datetime

In [ ]:
# Source folder where the files are currently located
source_folder = "inbox"

# Destination folder where all files will be moved
destination_folder = "data"

# Create the destination folder if it doesn't exist
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# Iterate through all the subdirectories in the source folder
for root, dirs, files in os.walk(source_folder):
    for file in files:
        # Check if the file is a JSON file
        if file.endswith(".json"):
            # Create the source file path
            source_file_path = os.path.join(root, file)

            # Create the destination file path by joining the destination folder path and the file name
            destination_file_path = os.path.join(destination_folder, file)

            # Handle filename conflicts by adding a number suffix
            counter = 1
            while os.path.exists(destination_file_path):
                filename, ext = os.path.splitext(file)
                new_filename = f"{filename}_{counter}{ext}"
                destination_file_path = os.path.join(destination_folder, new_filename)
                counter += 1

            # Move the file from the source location to the destination location
            shutil.move(source_file_path, destination_file_path)

Step 2: Merge all files and dictonaries into one.

In [ ]:

# Get the folder path for "data"
folder_path = os.path.join(os.getcwd(), "data")

# List all the files in the "data" folder
file_list = os.listdir(folder_path)

# Filter JSON files from the list
json_files = [file for file in file_list if file.endswith(".json")]

# Dictionary to hold all the messages from different files
all_messages_data = {}

# Counter to keep track of the messages keys
counter = 1

# Iterate through each JSON file and create a new dictionary for each set of messages
for json_file in json_files:
    json_file_path = os.path.join(folder_path, json_file)
    with open(json_file_path, "r") as file:
        data = json.load(file)

    messages = data.get("messages", [])

    # Create a new key (messages1, messages2, messages3, etc.) for each set of messages
    key = f"messages{counter}"
    all_messages_data[key] = messages

    counter += 1

# Save all the messages data into a new JSON file
merged_file_path = os.path.join(os.getcwd(), "merged_data.json")
with open(merged_file_path, "w") as merged_file:
    json.dump(all_messages_data, merged_file, indent=4)

print("Merged data saved to 'merged_data.json'.")

Step 3: reversed the data and saved them into rows and columns

In [ ]:
# Read the data from the JSON file
with open("merged_data.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# Create a CSV file to store the processed data
with open("test2.csv", "w", newline='', encoding="utf-8") as csv_file:
    csv_writer = csv.writer(csv_file)

    # Write the header row
    header = ["Dictionary Name"]
    max_messages = max(len(messages) for messages in data.values())
    for i in range(1, max_messages + 1):
        header.extend([f"sender_name{i}", f"timestamp_ms{i}", f"content{i}"])
    csv_writer.writerow(header)

    # Process each dictionary in the merged data
    for dictionary_name, messages in data.items():
        messages = list(reversed(messages))  # Convert to a list and reverse the order of messages
        row = [dictionary_name]
        for message in messages:
            sender_name = message.get("sender_name", "")
            timestamp_ms = message.get("timestamp_ms", "")
            content = json.dumps(message.get("content", ""), ensure_ascii=False)
            row.extend([sender_name, timestamp_ms, content])

        # Pad the row with empty strings if the dictionary has fewer messages
        padding = [''] * ((max_messages - len(messages)) * 3)
        row.extend(padding)

        csv_writer.writerow(row)

print("Data processed and saved to 'test2.csv'.")


Step 4: converting timestamps to date and time

In [ ]:

def convert_timestamp_to_datetime(timestamp):
    try:
        # Convert the timestamp to a datetime object
        timestamp_ms = int(timestamp)  # Convert timestamp to integer if it's a string
        datetime_obj = datetime.datetime.fromtimestamp(timestamp_ms / 1000)  # Convert from milliseconds to seconds
        return datetime_obj.strftime('%Y-%m-%d %H:%M:%S')  # Format as desired
    except ValueError:
        return timestamp  # Return the original value if the conversion fails

# Read the CSV file and load its data
with open('test2.csv', 'r', newline='', encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file)
    data = [row for row in csv_reader]

# Identify timestamp columns based on their labels and convert them
header = data[0]
timestamp_indices = [i for i, column in enumerate(header) if column.startswith('timestamp_ms')]
for row in data[1:]:
    for i in timestamp_indices:
        row[i] = convert_timestamp_to_datetime(row[i])

# Write the updated data back to the CSV file
with open('output2.csv', 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(data)

print("Timestamps converted to regular date and time format and saved to 'output2.csv'.")

Step 5: Extracting the relevant dated data, from August 1 to August 30

In [ ]:
# Read the data from the CSV file
data = []
with open('output2.csv', 'r', newline='', encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        data.append(row)

# Get the header row and find the index of the first timestamp column (timestamp_ms1)
header = data[0]
timestamp_index = header.index('timestamp_ms1')


import datetime

# Define the start and end dates
start_date = datetime.date(2023, 8, 1)
end_date = datetime.date(2023, 8, 30)

# Filter the data to keep only the rows where the first timestamp is within the specified date range
filtered_data = [row for row in data[1:] if start_date <= datetime.datetime.strptime(row[timestamp_index], '%Y-%m-%d %H:%M:%S').date() <= end_date]

# Filter the data to keep only the rows where the first timestamp is on or after July 1, 2023
#filtered_data = [row for row in data[1:] if datetime.datetime.strptime(row[timestamp_index], '%Y-%m-%d %H:%M:%S').date() >= datetime.date(2023, 8, 23)]

# Write the filtered data to a new CSV file
with open('filtered_output.csv', 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(header)  # Write the header row
    csv_writer.writerows(filtered_data)

print("Messages with the first timestamp on or after August 1, 2023, are saved to 'filtered_output.csv'.")

Step 6: Filter out all conversations started by healthtracka, we want to deal with conversations started by leads

In [ ]:
# Read the data from the CSV file
data = []
with open('filtered_output.csv', 'r', newline='', encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        data.append(row)

# Get the header row and find the index of the sender_name1 column
header = data[0]
sender_name1_index = header.index('sender_name1')

# Filter the data to keep only the rows where sender_name1 is not "healthtracka"
filtered_data = [row for row in data[1:] if row[sender_name1_index] != "Healthtracka"]

# Write the filtered data to a new CSV file
with open('filtered_output2.csv', 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(header)  # Write the header row
    csv_writer.writerows(filtered_data)

print("Entries with sender_name1 as 'Healthtracka' are removed. Filtered data is saved to 'filtered_output2.csv'.")

# Data Analysis
Step 1: Finding out the number of text messages Healthtracka got during 1 Aug to 30 Aug

In [ ]:
import csv
import datetime
import matplotlib.pyplot as plt

In [ ]:
# Read the data from the CSV file
data = []
with open('filtered_output2.csv', 'r', newline='', encoding='utf-8') as csv_file:
     csv_reader = csv.reader(csv_file)
     for row in csv_reader:
        data.append(row)

 # Get the header row and find the index of the first timestamp column (timestamp_ms1)
header = data[0]
timestamp_index = header.index('timestamp_ms1')

# Create a dictionary to store the count of messages for each date
messages_count_by_date = {}

 # Count the messages for each date
for row in data[1:]:
    timestamp_str = row[timestamp_index]
    date = datetime.datetime.strptime(timestamp_str, '%Y-%m-%d %H:%M:%S').date()
    if date in messages_count_by_date:
       messages_count_by_date[date] += 1
    else:
       messages_count_by_date[date] = 1

# Plot the graph
dates = list(messages_count_by_date.keys())
counts = list(messages_count_by_date.values())

plt.figure(figsize=(10, 6))
plt.bar(dates, counts)
plt.xlabel('Date')
plt.ylabel('Number of new DMs')
plt.title('Number of new DMs received each date')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Step 2: Analyzing the chat messages

In [ ]:
# Read the data from the CSV file
data = []
with open('filtered_output2.csv', 'r', newline='', encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        data.append(row)

# Get the header row and find the indices of columns
header = data[0]
content_indices = [i for i, col in enumerate(header) if col.startswith('content')]

# Initialize a dictionary to store the number of filled columns for each conversation
conversation_filled_counts = {}

# Process each row in the data
for row in data[1:]:
    # Calculate the number of filled columns for the current conversation
    num_filled_columns = sum(1 for content_index in content_indices if row[content_index].strip() != "")

    # Find the sender_name1 for the current row
    sender_name1 = row[1]  # Replace 1 with the correct index of sender_name1 in your CSV file

    # Update the conversation_filled_counts dictionary
    if sender_name1 != "Healthtracka":
        if sender_name1 not in conversation_filled_counts:
            conversation_filled_counts[sender_name1] = []

        conversation_filled_counts[sender_name1].append(num_filled_columns)

# Calculate the total filled columns for each sender_name1
sender_names = []
total_filled_columns = []
for sender, filled_counts in conversation_filled_counts.items():
    sender_names.append(sender)
    total_filled_columns.append(sum(filled_counts))

# Calculate the average, minimum, and maximum total filled columns
sum_filled_columns = sum(total_filled_columns)
average_filled_columns = sum(total_filled_columns) / len(total_filled_columns)
min_filled_columns = min(total_filled_columns)
max_filled_columns = max(total_filled_columns)

# Print the results
print(f"Total Number of chat messages received : {sum_filled_columns}")
print(f"Average number of messages per Sender: {average_filled_columns:.2f}")
print(f"Minimum number of messages per Sender: {min_filled_columns}")
print(f"Maximum number of messages per Sender: {max_filled_columns}")


Step 3: Listing out relevent sentences/messages

In [ ]:
import csv
import nltk

nltk.download('punkt')


def is_question(sentence):
    # Check if any question word is present in the sentence
    question_words = {"health", "checkup", "package", "test", "price", "for", "it", "to", "i'll", "i", "much", "cost",
                      "based", " in ", "stay", "body", " full ", "liver",
                      "kidney", "heart", "my", " when ",
                      "check", "what", "will", "male", "fertility", "based", "pls", "please", "in", "need", "know",
                      "can", "is", "or", "if", "when", "where", "who", "why", "how", "are", "do", "like", "book",
                      "want", "full", "liver", "all", "std", "sti", "blood", "body", "check", "checkup", "is"}
    words = nltk.word_tokenize(sentence.lower())
    return any(word in question_words for word in words)


# Read the data from the CSV file
data = []
with open('filtered_output2.csv', 'r', newline='', encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        data.append(row)

# Get the header row and find the indices of columns
header = data[0]
sender_name_indices = [i for i, col in enumerate(header) if col.startswith('sender_name')]
content_indices = [i for i, col in enumerate(header) if col.startswith('content')]

# Prepare the list to store questions with corresponding sender_name
questions_with_senders = []

# Initialize the total number of sentences counter
total_sentences_count = 0

# Process each row in the data
for row in data[1:]:
    for sender_index, content_index in zip(sender_name_indices, content_indices):
        sender_name = row[sender_index]
        content = row[content_index]

        # Check if the sender_name is not "healthtracka"
        if sender_name != "Healthtracka":
            # Tokenize content into sentences
            sentences = nltk.sent_tokenize(content)

            # Increment the total sentences count
            total_sentences_count += len(sentences)

            # Print the content of each column with its index for the current conversation
            # print(f"Conversation with {sender_name} has {len(sentences)} sentences:")
            # for g, sentence in enumerate(sentences, 1):
            #     print(f"{g}. {sentence}")

            for sentence in sentences:
                if is_question(sentence):
                    questions_with_senders.append((sender_name, sentence))

# Print the total number of sentences
print(f"Total number of sentences: {total_sentences_count}")

# Print the questions with corresponding sender_name for each content column
print("Questions by Senders:")
for i, (sender, question) in enumerate(questions_with_senders, 1):
    print(f"{i}. Sender: {sender}, Question: {question}")


Step 4: Classifying sentences

In [ ]:
import csv
import nltk

nltk.download('punkt')


def is_question(sentence):
    # Check if any question word is present in the sentence
    question_words = {"health", "checkup", "package", "test", "price", "for", "it", "to", "i'll", "i", "much", "cost", "based",
                      " in ", "stay", "body", " full ", "liver",
                      "kidney", "heart", "my", "am",
                      "check", "what", "will", "male", "fertility", "based", "pls", "please", "in", "need", "know",
                      "can", "is", "or", "if", "when", "where", "who", "why", "how", "are", "do", "like", "book",
                      "want", "full", "liver", "all", "std", "sti", "blood", "body", "check", "checkup", "is"}
    words = nltk.word_tokenize(sentence.lower())
    return any(word in question_words for word in words)


# Read the data from the CSV file
data = []
with open('filtered_output2.csv', 'r', newline='', encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        data.append(row)

# Get the header row and find the indices of columns
header = data[0]
sender_name_indices = [i for i, col in enumerate(header) if col.startswith('sender_name')]
content_indices = [i for i, col in enumerate(header) if col.startswith('content')]

# Prepare the list to store questions with corresponding sender_name
questions_with_senders = []
questions_with_senders1 = []
questions_with_senders2 = []
questions_with_senders3 = []
questions_with_senders4 = []
questions_with_senders5 = []
questions_with_senders6 = []
questions_with_senders7 = []

# Process each row in the data
for row in data[1:]:
    for sender_index, content_index in zip(sender_name_indices, content_indices):
        sender_name = row[sender_index]
        content = row[content_index]

        # Check if the sender_name is not "healthtracka"
        if sender_name != "Healthtracka":
            # Tokenize content into sentences
            sentences = nltk.sent_tokenize(content)
            for sentence in sentences:
                if is_question(sentence) and any(word in sentence.lower() for word in
                                                 ["price", "much", "spend", "charge", "cost", "insurance", "free", "HMO", "pay",
                                                  "payment"]):
                    questions_with_senders.append((sender_name, sentence))

                if is_question(sentence) and any(word in sentence.lower() for word in ["based", " in ", "stay", "live"]):
                    questions_with_senders1.append((sender_name, sentence))

                if is_question(sentence) and any(
                        word in sentence.lower() for word in ["body", " full ", "liver", "kidney", "heart", "check", "general", "health"]):
                    questions_with_senders2.append((sender_name, sentence))

                if is_question(sentence) and any(
                        word in sentence.lower() for word in [" std ", " sti ", "hpv",  "hiv", "herpes", "sexual health", "sexual"]):
                    questions_with_senders3.append((sender_name, sentence))

                if is_question(sentence) and any(
                        word in sentence.lower() for word in ["couple", "wedding", "marriage", "fertility"]):
                    questions_with_senders4.append((sender_name, sentence))

                if is_question(sentence) and any(word in sentence.lower() for word in
                                                 ["what", "type", "which", "service","if", "do you", "asking", "offer", "know", "more", "i have","does", "feeling", "pain", "my"]):
                    questions_with_senders5.append((sender_name, sentence))

                if is_question(sentence) and any(
                        word in sentence.lower() for word in ["I would", "like", "I want", "to book", "want", "yes", "place", "can", "to test", "can i", "want", "would", "do i", "interested"]):
                    questions_with_senders6.append((sender_name, sentence))

                if is_question(sentence) and any(word in sentence.lower() for word in ["I have",
                                                                                       " will ", "are scheduled", "today", "booked", "have booked", "scheduled",]) and "cost" not in sentence.lower() and "like" not in sentence.lower():
                    questions_with_senders7.append((sender_name, sentence))

# Save the output to a new CSV file
with open('Statement_Classification.csv', 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)

    # Write headers for the first category
    writer.writerow(["Statement Related to 'Price', 'Much', or 'Cost':"])
    writer.writerow(["Index", "Question"])
    for i, (_, question) in enumerate(questions_with_senders, 1):
        writer.writerow([i, question])

    # Empty row for spacing between categories
    writer.writerow([])

    # Write headers for the second category
    writer.writerow(["Statement Related to 'in', 'at', or 'stay':"])
    writer.writerow(["Index", "Question"])
    for g, (_, question1) in enumerate(questions_with_senders1, 1):
        writer.writerow([g, question1])

    # Empty row for spacing between categories
    writer.writerow([])

    # Write headers for the third category
    writer.writerow(["Statement Related to 'full', 'body', or 'check':"])
    writer.writerow(["Index", "Question"])
    for j, (_, question2) in enumerate(questions_with_senders2, 1):
        writer.writerow([j, question2])

    # Empty row for spacing between categories
    writer.writerow([])

    # Write headers for the fourth category
    writer.writerow(["Statement Related to 'std', 'sti', or 'sexual':"])
    writer.writerow(["Index", "Question"])
    for k, (_, question3) in enumerate(questions_with_senders3, 1):
        writer.writerow([k, question3])

    # Empty row for spacing between categories
    writer.writerow([])

    # Write headers for the fifth category
    writer.writerow(["Statement Related to 'couple', 'wedding', 'marriage':"])
    writer.writerow(["Index", "Question"])
    for l, (_, question4) in enumerate(questions_with_senders4, 1):
        writer.writerow([l, question4])

    # Empty row for spacing between categories
    writer.writerow([])

    # Write headers for the sixth category
    writer.writerow(["Statement Related to 'what', 'type', 'which':"])
    writer.writerow(["Index", "Question"])
    for m, (_, question5) in enumerate(questions_with_senders5, 1):
        writer.writerow([m, question5])

    # Empty row for spacing between categories
    writer.writerow([])

    # Write headers for the seventh category
    writer.writerow(["Statement Related to 'I would', 'I want', 'to book':"])
    writer.writerow(["Index", "Question"])
    for n, (_, question6) in enumerate(questions_with_senders6, 1):
        writer.writerow([n, question6])

    # Empty row for spacing between categories
    writer.writerow([])

    # Write headers for the eighth category
    writer.writerow(["Statement Related to 'will', 'I have' :"])
    writer.writerow(["Index", "Question"])
    for o, (_, question7) in enumerate(questions_with_senders7, 1):
        writer.writerow([o, question7])
